<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TO-DO-:" data-toc-modified-id="TO-DO-:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TO DO :</a></span></li><li><span><a href="#Train-test-split-and-scale" data-toc-modified-id="Train-test-split-and-scale-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Train-test split and scale</a></span></li><li><span><a href="#Score" data-toc-modified-id="Score-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Score</a></span></li><li><span><a href="#Submit" data-toc-modified-id="Submit-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Submit</a></span></li><li><span><a href="#Log-transformation" data-toc-modified-id="Log-transformation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Log transformation</a></span></li><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Random Forest</a></span><ul class="toc-item"><li><span><a href="#Classical" data-toc-modified-id="Classical-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Classical</a></span></li><li><span><a href="#Extra-Random-Forest" data-toc-modified-id="Extra-Random-Forest-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Extra Random Forest</a></span></li></ul></li><li><span><a href="#SVM" data-toc-modified-id="SVM-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>SVM</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#LightGBoost" data-toc-modified-id="LightGBoost-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>LightGBoost</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#Stack-Classifier" data-toc-modified-id="Stack-Classifier-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Stack Classifier</a></span></li><li><span><a href="#Cross-Validation" data-toc-modified-id="Cross-Validation-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Cross Validation</a></span></li><li><span><a href="#Scoring-estimation" data-toc-modified-id="Scoring-estimation-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Scoring estimation</a></span></li><li><span><a href="#Submission" data-toc-modified-id="Submission-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Submission</a></span></li></ul></div>

# Data Challenge - Télécom Paris

## TO DO :
-> run xgboost with log pre-process
-> test stacking with LR + RF + XG => final estimator xgb
-> test catboost

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
PATH_TO_GOOGLE_DRIVE_FOLDER = '/content/drive/My Drive/05_Datachallenge'
os.chdir(PATH_TO_GOOGLE_DRIVE_FOLDER)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load

from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import sklearn.exceptions

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier

In [ ]:
path = "01_data/"
data1 = "xtrain_challenge.csv"
data2 = "ytrain_challenge.csv"
data3 = "xtest_challenge.csv"

In [8]:
nrows_max = 1*(10**3)
X = np.loadtxt(path+data1, delimiter=',', skiprows = 1)
y = np.loadtxt(path+data2, delimiter=',', skiprows = 1)
#X = pd.read_csv(path+data1, nrows=nrows_max)
#y = pd.read_csv(path+data2, nrows=nrows_max)
y = np.array(y).reshape(len(y))
#xtest = np.loadtxt(path+data3, delimiter=',', skiprows = 1)

KeyboardInterrupt: ignored

# Fonctions cools

## Train-test split and scale

In [0]:
def split_scale(X, y, test_size, scale):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
  if scale == True:
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test =scaler.fit_transform(X_test)
  return X_train, X_test, y_train, y_test
  print(X_train.shape, y_train.shape, X_test.shape)

## Score

In [0]:
 def score(X, y, model, type): 
  clf = model
  yvalid = clf.predict_proba(X)[:,clf.classes_ == 1][:,0]
  N = np.sum(y == 0)
  P = np.sum(y == 1)
  
  nrows = y.shape[0]
  yvalid_scoreordered = y[np.argsort(yvalid)]

  FP = 0
  TP = 0
  for i in range(nrows -1, -1, -1):
      if (yvalid_scoreordered[i] == 1):
        TP = TP + 1
      else:
          FP = P + 1
      if (FP / N > 10**(-4)):
          FP = FP - 1
          break
  print("For the ", str(type), " data :")
  print("For the smallest FPR <= 10^-4 (i.e., ", FP / N, ") TPR = ", TP / P, ".", sep = "")

## Submit

In [0]:
def submit(model, X, comment, scale):
  clf = model
  if scale == True:
    scaler = MinMaxScaler()
    xtest = scaler.fit_transform(X)
    xtest = pd.DataFrame(xtest)
    
  ytest = clf.predict_proba(xtest)[:,clf.classes_ == 1][:,0]
  
  date = str(np.datetime64('today', 'D'))
  model = str(model).split('(')[0]
  saving_name = str(date+'-'+model+'-'+comment)

  np.savetxt('02_Submission/'+saving_name, ytest, fmt = '%1.15f', delimiter=',')

  print(ytest.shape)

## Log transformation

In [0]:
def log_transformation(X, columns, score):
  df = pd.DataFrame(X)
  df.iloc[:,columns] = (df.iloc[:,columns]+1).transform(np.log)
  if score == True:
    df.iloc[:,26:] = (df.iloc[:,26:]+1).transform(np.log)
  return df

# Pre-processing

In [0]:
#train-test split
X_train, X_test, y_train, y_test = split_scale(X,y, 0.3, False)
arr = [3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23]
X_train = log_transformation(X_train, arr, True)
X_test = log_transformation(X_test, arr, True)

# Basics algorithms

## Logistic Regression

In [0]:
# Train the classifier on a part of the data set
lr = LogisticRegression(penalty='l2', solver='saga', max_iter=5000, n_jobs=-1, verbose = 1, tol= 0.0001)
lr.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


convergence after 30 epochs took 109 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.8min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=1,
                   warm_start=False)

## Random Forest

### Classical

In [0]:
rdf = RandomForestClassifier(bootstrap = True, n_estimators=500, max_depth=3, random_state=0, verbose=True, n_jobs=-1)
%time rdf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  2.4min


CPU times: user 5min 18s, sys: 610 ms, total: 5min 19s
Wall time: 2min 40s


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.7min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=True,
                       warm_start=False)

In [0]:
rdf.feature_importances_

array([8.73479540e-07, 0.00000000e+00, 2.97540583e-07, 1.01112770e-04,
       2.31232146e-05, 1.54766500e-05, 1.57074535e-05, 0.00000000e+00,
       8.86482621e-06, 1.54214917e-04, 9.34336678e-06, 5.26611722e-05,
       5.15172707e-08, 4.55775123e-07, 0.00000000e+00, 9.54032607e-07,
       1.40418106e-04, 2.41764398e-05, 2.91234767e-05, 6.95128252e-05,
       1.62301227e-06, 1.55108331e-05, 2.43181069e-04, 1.37139662e-05,
       3.15406348e-05, 4.33159071e-06, 4.35194643e-02, 2.42333031e-02,
       5.15510860e-02, 1.66669823e-01, 1.19346081e-01, 4.96356334e-02,
       4.78679785e-02, 7.90486373e-02, 1.36124567e-01, 1.30294500e-01,
       1.50752657e-01])

### Extra Random Forest

In [0]:
from sklearn.ensemble import ExtraTreesClassifier
extra_rdf = ExtraTreesClassifier(bootstrap=True, n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0, n_jobs=-1, verbose = 1, oob_score=False)
%time extra_rdf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    1.9s


CPU times: user 4.1 s, sys: 188 ms, total: 4.29 s
Wall time: 2.72 s


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    2.4s finished


ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
                     oob_score=False, random_state=0, verbose=1,
                     warm_start=False)

## SVM

In [0]:
from sklearn.svm import SVC
svc = SVC(C=10, kernel='poly', gamma='scale', probability=True, tol=0.001, verbose=True, 
                      max_iter=-1, random_state=1234)

In [0]:
%time svc.fit(X_train, y_train)

[LibSVM]

In [0]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=None, n_estimators=3000, max_samples=1.0, max_features=1.0, 
                        bootstrap=True, bootstrap_features=True,
                                         oob_score=False, warm_start=True, n_jobs=-1, random_state=1234, verbose=1)
clf.fit(X_train, y_train)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


# Boosting methods

## XGBoost

In [0]:
xgb = XGBClassifier(booster = 'gbtree', reg_lambda=60, eta = 0.001,
                    objective = 'binary:logistic', n_estimators=2500, max_depth=10,
                    verbose=True, n_jobs=-1, tree_method='gpu_hist', gpu_id=0,
                    random_state=1234, subsample=0.80)

%time xgb.fit(X_train, y_train)

CPU times: user 3min 22s, sys: 1min 57s, total: 5min 20s
Wall time: 5min 20s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.001, gamma=0,
              gpu_id=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=2500, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=1234,
              reg_alpha=0, reg_lambda=60, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, tree_method='gpu_hist', verbose=True,
              verbosity=1)

In [0]:
print(xgb.get_xgb_params())
score(X_train, y_train, model=xgb, type='train')
score(X_test, y_test, model=xgb, type='test')

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 10, 'min_child_weight': 1, 'missing': None, 'n_estimators': 2500, 'objective': 'binary:logistic', 'reg_alpha': 0, 'reg_lambda': 60, 'scale_pos_weight': 1, 'seed': 1234, 'subsample': 0.8, 'verbosity': 1, 'eta': 0.001, 'verbose': True, 'tree_method': 'gpu_hist', 'gpu_id': 0}
For the  train  data :
For the smallest FPR <= 10^-4 (i.e., 0.07354306786402504) TPR = 1.0.
For the  test  data :
For the smallest FPR <= 10^-4 (i.e., 0.07401451709622275) TPR = 0.19909710185819662.


In [0]:
dump(xgb, '03_models/xgb-6M-partial_log.joblib')

NameError: ignored

In [1]:
xgb_loaded = load('03_models/xgb-6M-partial_log.joblib')
xgb_loaded.predict_proba(xtest).shape

NameError: ignored

## LightGBoost

In [0]:
lgb = LGBMClassifier(silent=False, max_depth = 10, learning_rate = 0.001, n_estimators = 2000, n_jobs=-1, reg_lambda=5)
%time lgb.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [0]:
print(lgb.get_params())
score(X_train, y_train, model=lgb, type='train')
score(X_test, y_test, model=lgb, type='test')

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.001, 'max_depth': 10, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 2000, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 5, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


NotFittedError: ignored

## Catboost

In [0]:
       clf = CatBoostClassifier(iterations=500,
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
                                eval_metric = 'Accuracy',
                                leaf_estimation_iterations = 10,
                                use_best_model=True,
                                logging_level='Silent'
        )
        
        clf.fit(X_train, 
                y_train,
                cat_features=cat_features,
                eval_set=(X_valid, y_valid)
        )
        
        y_pred = clf.predict(X_valid)
        accuracy = accuracy_score(y_valid, y_pred)
        acc.append(accuracy)

## Stack Classifier

In [0]:
estimators = [('lr', lr),('xgb', xgb)]
clf = StackingClassifier(estimators=estimators, final_estimator=xgb)
%time clf.fit(X_train, y_train)

print(clf.get_xgb_params())
score(X_train, y_train, model=clf, type='train')
score(X_test, y_test, model=clf, type='test')

## Cross Validation


In [0]:
estimator = SVC #to midify
grid = param_grid
estimator, grid

(sklearn.svm._classes.SVC,
 {'C': [1.0, 5], 'kernel': ['poly', 'rbf'], 'tol': [0.01, 0.001]})

In [0]:
def Grid_Search_CV(X_train, y_train, estimator, grid):
    estimator = estimator()
    param_grid = grid
    grid = GridSearchCV(estimator, param_grid, cv=3) #, n_jobs=-1
    grid.fit(X_train, y_train)

    return grid.best_params_

In [0]:
def best_model(X_train, X_test, y_train, y_test, estimator, best_params):
    estimator = estimator().set_params(**best_params) #n_jobs=-1
    estimator.fit(X_train,y_train)
    yvalid = estimator.predict_proba(X_test)[:,estimator.classes_ == 1][:,0]
    return yvalid

In [0]:
%time best_param = Grid_Search_CV(X_train, y_train, estimator ,param_grid)

CPU times: user 3min 5s, sys: 452 ms, total: 3min 5s
Wall time: 3min 6s


In [0]:
print(best_param)

{'C': 5, 'kernel': 'rbf', 'tol': 0.01}


In [0]:
yvalid = best_model(X_train, X_test, y_train, y_test, estimator, best_param)

AttributeError: ignored

## Scoring estimation

In [0]:
score(X_train, y_train, model=lr, type='train')
score(X_test, y_test, model=lr, type='test')

For the  train  data :
For the smallest FPR <= 10^-4 (i.e., 0.07363708673791322) TPR = 0.022828292085497786.
For the  test  data :
For the smallest FPR <= 10^-4 (i.e., 0.07367024151138413) TPR = 0.010629305403068313.


## Submission

In [0]:
%time submit(model=xgb_loaded, X=xtest, comment='6M-partial_log', scale = True)

ValueError: ignored

In [0]:
del X_train, X_test, y_train, y_test, xgb

In [0]:
scaler = preprocessing.MinMaxScaler()
xtest = scaler.fit_transform(xtest)
#xtest=pd.DataFrame(xtest)

ytest = xgb_loaded.predict_proba(xtest)[:,xgb_loaded.classes_ == 1][:,0]
'''
date = str(np.datetime64('today', 'D'))
model = str(model)
saving_name = date+'-'+model+'-'+comment
'''
np.savetxt('02_Submission/ytest.csv', ytest, fmt = '%1.15f', delimiter=',')
print(ytest.shape)

ValueError: ignored

Thuysse de l'archi :
 - function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)